# Open Source GIS Pre-Processing Tutorial
#### First, set the file paths for inputs and outputs

Throughout this exercise, we will use python variables to store directory paths and other variables. However, we will call all GIS Pre-processing functionality as though it were on the command line. This is done by adding `!` syntax before the command-line syntax, to execute the line using bash.

In this cell, Python variables are created that point to the file paths of the test-case data and an output directory is defined to store the data created by these tools.

In [6]:
### 确定工作路径
# Import python core modules
import os
import shutil
import whitebox

# Set root directory for GIS lesson
gis_data_folder = "/home/fengx20/project/hydro/test2/DATA/gis/"

# Change the directory to the GIS_Training directory and get current working directory
os.chdir(gis_data_folder)
cwd = os.getcwd()

# Set paths to known input and output directories and files
data_folder = os.path.join(cwd, 'data')
script_folder = os.path.join(cwd, 'wrfhydro_gis')

in_geogrid = os.path.join(data_folder, 'geo_em.d01.nc')
output_folder = os.path.join(cwd, 'outputs')

# Clear any outputs from previous runs by deleting (if necessary) and re-creating the output directory
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)  # 清空outputs中文件
os.mkdir(output_folder)

In [7]:
# Print information to screen for reference
### 根据geo_em.d01.nc文件创建shp文件
print('Command to run:\n')
print('python Create_Domain_Boundary_Shapefile.py \\\n\t -i {0} \\\n\t -o {1}\n'.format(in_geogrid, output_folder))

# Run the script with required parameters
# os.system('python Create_Domain_Boundary_Shapefile.py -i {} -o {}').format(in_geogrid, output_folder)
os.chdir(script_folder)
! python Create_Domain_Boundary_Shapefile.py -i {in_geogrid} -o {output_folder}


Command to run:

python Create_Domain_Boundary_Shapefile.py \
	 -i /home/fengx20/project/hydro/test2/DATA/gis/data/geo_em.d01.nc \
	 -o /home/fengx20/project/hydro/test2/DATA/gis/outputs

Script initiated at Mon Oct 31 20:23:08 2022
WPS netCDF projection identification initiated...
    Map Projection: Lambert Conformal Conic
    Using MOAD_CEN_LAT for latitude of origin.
    Using Standard Parallel 2 in Lambert Conformal Conic map projection.
    Geo-referencing step completed without error in  0.03 seconds.
    Created projection definition from input NetCDF GEOGRID file.
      ESRI Shapefile driver is available.
  Done producing output vector polygon shapefile in  0.01 seconds
  Output shapefile: /home/fengx20/project/hydro/test2/DATA/gis/outputs/geo_em.d01_boundary.shp
Process completed in 0.09 seconds.


In [8]:
## 2. Build GeoTiff raster from a WPS Geogrid file
# Define the variable to export to raster
in_var = "HGT_M"

# Define the output raster file using variable name defined above
out_file = os.path.join(output_folder, f'{in_var}.tif')

# Print information to screen for reference
print('Command to run:\n')
print('python Build_GeoTiff_From_Geogrid_File.py \\\n\t -i {0} \\\n\t -v {1} \\\n\t -o {2}\n'.format(in_geogrid, in_var, out_file))

# Run the script with required parameters
os.system('cd {}'.format(script_folder))
! python Build_GeoTiff_From_Geogrid_File.py -i {in_geogrid} -v {in_var} -o {out_file}

Command to run:

python Build_GeoTiff_From_Geogrid_File.py \
	 -i /home/fengx20/project/hydro/test2/DATA/gis/data/geo_em.d01.nc \
	 -v HGT_M \
	 -o /home/fengx20/project/hydro/test2/DATA/gis/outputs/HGT_M.tif

Script initiated at Mon Oct 31 20:23:09 2022
Using default variable name: HGT_M
Input WPS Geogrid or Fulldom file: /home/fengx20/project/hydro/test2/DATA/gis/data/geo_em.d01.nc
Input netCDF variable name: HGT_M
Output raster file: /home/fengx20/project/hydro/test2/DATA/gis/outputs/HGT_M.tif
WPS netCDF projection identification initiated...
    Map Projection: Lambert Conformal Conic
    Using MOAD_CEN_LAT for latitude of origin.
    Using Standard Parallel 2 in Lambert Conformal Conic map projection.
    Geo-referencing step completed without error in  0.02 seconds.
    X-dimension: 'west_east'.
    Y-dimension: 'south_north'.
    Reversing order of dimension 'south_north'
    Time dimension found: 'Time'.
      Time dimension size = 1.
    Dimensions and indices or slices on tho

# 生成路径文件

The `Build_Routing_Stack.py` script is a program to build the full set of hydrologically-processed routing grids and additional data required by WRF-Hydro. This is the main utility for performing WRF-Hydro GIS pre-processing. The required inputs are the domain file (WPS geogrid file), desired routing grid resolution as a function of geogrid resolution, and other options and parameter values. The output will be a "routing stack" zip file with WRF-Hydro domain and parameter files.

• Required Parameters:<br>
&emsp;`-i` -WRF/WPS GEOGRID file (geo_em.d0*.nc)<br>
&emsp;`-d` -High-resolution Elevation raster file (Esri GRID, GeoTIFF, VRT, etc.)<br>
&emsp;`-R` -Regridding Factor – nesting relationship of routing:land surface model grid cells<br>
&emsp;`-t` -Minimum basin area threshold (in routing grid cells)<br>
&emsp;`-o` -Output ZIP File containing all script outputs<br>

• Optional Parameters:<br>
&emsp;`--CSV` -Station Locations location file (.csv)<br>
&emsp;`-b` -Option to mask channel grids not contributing to provided station locations<br>
&emsp;`-r` -Reach based (Muskingum / Muskingum-Cunge) routing option<br>
&emsp;`-l` -Lake Polygons (polygon feature class or .shp)<br>
&emsp;`-O` -OVROUGHRTFAC – Multiplier on Manning's roughness for overland flow. default=1.0<br>
&emsp;`-T` -RETDEPRTFAC – Multiplier on maximum retention depth before flow is routed as overland flow. default=1.0<br>
&emsp;-LKSATFAC – (script global variable) Multiplier on saturated hydraulic conductivity in lateral flow direction. default=1000.0<br>
&emsp;`--starts` -Path to point shapefile or feature class containing channel initiation locations (overrides `-t` parameter)<br>
&emsp;`--gw` -Path to polygon shapefile or feature class containing prescribed groundwater basin locations<br>

In [13]:
## 3. Building the hydrologic routing grids, aka the "routing stack"
import Build_Routing_Stack

# Define script input parameters using python variables
in_geogrid = os.path.join(data_folder, 'geo_em.d01.nc')
lakes = os.path.join('/home/fengx20/project/hydro/test2/DATA/gis/data/edm/HydroLAKES_polys_v10_shp', 'HydroLAKES_polys_v10.shp')
csv = os.path.join(data_folder, 'sta.csv')
in_dem = os.path.join(data_folder, 'edm', 'as_dem_3s.tif')
regrid_factor = 10
routing_cells = 200
out_zip = os.path.join(output_folder, 'test.zip')

# Print information to screen for reference
print('Command to run:\n')
print('python Build_Routing_Stack.py \\\n\t -i {0} \\\n\t -l {1} \\\n\t --CSV {2} \\\n\t -d {3} \\\n\t -R {4} \\\n\t -t {5} \\\n\t -o {6}\n'.format(in_geogrid, lakes, csv, in_dem, regrid_factor, routing_cells, out_zip))

# Run the script with required parameters
! python Build_Routing_Stack.py -i {in_geogrid} -l {lakes} --CSV {csv} -d {in_dem} -R {regrid_factor} -t {routing_cells} -o {out_zip}

Command to run:

python Build_Routing_Stack.py \
	 -i /home/fengx20/project/hydro/test2/DATA/gis/data/geo_em.d01.nc \
	 -l /home/fengx20/project/hydro/test2/DATA/gis/data/edm/HydroLAKES_polys_v10_shp/HydroLAKES_polys_v10.shp \
	 --CSV /home/fengx20/project/hydro/test2/DATA/gis/data/sta.csv \
	 -d /home/fengx20/project/hydro/test2/DATA/gis/data/edm/as_dem_3s.tif \
	 -R 10 \
	 -t 200 \
	 -o /home/fengx20/project/hydro/test2/DATA/gis/outputs/test.zip

Script initiated at Mon Oct 31 20:28:27 2022
  Parameter values that have not been altered from script default values:
    Using default basin mask setting: False
    Using default reach-based routing setting: False
    Using default regridding factor: 10
    Using default stream initiation threshold: 200
    Using default OVROUGHRTFAC parameter value: 1.0
    Using default RETDEPRTFAC parameter value: 1.0
  Values that will be used in building this routing stack:
    Input WPS Geogrid file: /home/fengx20/project/hydro/test2/DATA/gis/data/ge

### 4. Understanding the outputs

The `Build_Routing_Stack.py` script creates a Zip archive of output files according to the options provided to the tool. There will be at least four netCDF files. The output Zip file may additionally include shapefiles (.shp and accompanying files) describing the geometry of modeled lakes or the vector stream network. Below is an alphabetically sorted list of gridded variables that are created by the `Build_Routing_Stack.py` tool.

Fulldom_hires.nc file. This file stores 2D gridded variables that describe the hydro routing grid:
  + CHANNELGRID - The channel grid. Channel pixels = 0, non-channel pixels = -9999. If the `-b` option is set to TRUE, the output will be masked to the gaged basins provided, where non-gaged channels are given a value of ‘-1’. If lake routing is activated, lake outflow points will be identified by the lake ID value.
  + FLOWACC – Flow accumulation grid. This grid gives the number of contributing cells for each cell in the domain. This grid is provided for convenience and is not read by WRF-Hydro.
  + FLOWDIRECTION – Flow direction grid. This grid gives the direction of flow using the D8 algorithm between each cell and the steepest downslope neighbor according to Jenson and Domingue (1988). The result is an integer grid with values ranging from 1 to 128.
  + frxst_pts – Gage location grid. If a forecast point CSV file is provided, the grid will have a cell identified at the location of each forecast point (gage) in the gage CSV file. If no input CSV gage location file is provided, this grid will be uniform with values of ‘-9999’. Gage pixels are numbered in the same way as the ‘basn_msk’ grid. NoData cells are given a value of ‘-9999’.
  + basn_msk – Forecast basins grid. If a CSV gage location file is provided, catchments are delineated from a point that is up to 3 pixels downstream of the gage coordinates. This distance can be modified by altering the ‘walker’ global variable in the ‘wrfhydro_functions.py’ script. If masking of the ‘CHANNELGRID’ is selected, this layer is the mask. Basins are numbered according to the values in the ‘FID’ field of the input gage CSV file. If no gage location file is provided, this grid will be uniform with values of ‘-9999’.
  + LAKEGRID – The lake grid. If a lake polygon shapefile is provided to the `-l` parameter, this grid will contain ID values for each lake that can be resolved on the routing grid. Otherwise, this grid will be uniform with values of -9999.
  + landuse – This is the same data as the ‘LU_INDEX’ variable in the GEOGRID file, but resampled using Nearest Neighbor assignment to the resolution of the routing grid. This grid is provided for convenience and is not read by WRF-Hydro.
  + LATITUDE – Grid of the latitude at the center of each grid cell, in a geographic coordinate system (WGS84).
  + LINKID – The channel ID grid. This grid provides a unique integer identifier for each channel segment that is defined in the ‘link’ variable of the ‘Route_Link.nc’ file and the ‘STRM_VAL’ field in the ‘streams.shp’ shapefile. The ‘LINKID’ grid will only be created if the option `-r` is TRUE.
  + LONGITUDE – Grid of longitude value at the center of each grid cell, in a geographic coordinate system (WGS84).
  + OVROUGHRTFAC – OVROUGHRTFAC parameter. Currently set to a default of 1.0. This default value may be changed by providing an alternate value to the `-O` parameter.
  + RETDEPRTFAC – RETDEPRTFAC (retention depth multiplier) parameter. Currently set to a default of 1.0. This default value may be changed by providing an alternate value `-T` parameter.
  + STREAMORDER – Stream order grid, calculated using the Strahler method (Strahler 1957).
  + TOPOGRAPHY – Elevation grid. The units of elevation are the same as the input elevation raster dataset (`-d INDEM`), which should be specified in meters (m) above sea level (ASL). This grid is derived from the elevation values in the input elevation raster, but has been resampled to the routing grid resolution, and pit-filled to remove depressions.

Other files:
  + GEOGRID_LDASOUT_Spatial_Metadata.nc  - This is a CF-netCDF format file that provides the spatial metadata associated with the GEOGRID variables, which contsitute the LSM grid. By default, no 2-dimensional grids are written to the file. This file may be used by WRF-Hydro for appending geospatial metadata to the land surface model output, if necessary.
  + GWBASINS.nc - This is a 2D netCDF file of the location of groundwater basins, regridded to the LSM grid resolution. NoData cells are given a value of ‘-9999’. This file is by default created using the ‘FullDom LINKID local basins’ method of defining groundwater basins.
  + GWBUCKPARM.nc - The 1D groundwater basin parameter file.
  + LAKEPARM.nc – Lake parameter table. This 1D netCDF format file is created if a lake shapefile is provided as input to the `-l` parameter. The table will contain a record for each lake feature in the + Fulldom_hires.nc ‘LAKEGRID’ variable, and contain derived and default parameters for each lake.
  + Route_Link.nc – The reach-based routing parameter file. This 1D netCDF format file is created if the `-r` parameter is TRUE. The file contains a record for each stream segment. The stream segments in this table are also identified by the unique ‘LINKID’ values in the ‘LINKID’ variable in the ‘Fulldom_hires.nc’ file, and values in the ‘STRM_VAL’ field of the output ‘streams.shp’ shapefile. This table contains derived and default stream segment parameters that are calculated based on the vector stream network and topology in the ‘streams.shp’ shapefile.
  + streams.* (ancillary) - Streams shapefile, containing one feature for each stream segment in the domain. This file is meant to accompany the ‘Route_Link.nc’ reach-based routing parameter file and Fulldom_hires.nc ‘LINKID’ variable. The ‘streams’ shapefile is only created when the option `-l` is used. The ‘STRM_VAL’ field is the unique identifier for each stream segment, and corresponds to the ‘link’ variable of the ‘Route_Link.nc’ file and the ‘LINKID’ variable in the ‘Fulldom_hires.nc’ file. The geometry of the stream segments in this shapefile informs many of the parameters in the ‘Route_Link.nc’ file.
  + lakes.* (ancillary) - Lakes shapefile, containing one feature for each reservoir in the simulation domain. This file is meant to accopmany the 'LAKEPARM.nc' reservoir parameter file. If `-r TRUE` is used, then Fulldom_hires.nc 'LAKEID' variable will contain -9999 values only. The geometry of reservoir objects informs many of the parameters in 'LAKEPARM.nc' file.


### 5. Examine outputs of GIS pre-processor

The `Examine_Outputs_of_GIS_Preprocessor.py` script is a tool used to create easy-to-use geospatial files for examining the resulting WRF-Hydro input files in a GIS. This script takes the output ZIP file generated using the Process Geogrid script (executed above) and creates a raster from each 2D variable in each WRF-Hydro input netCDF file. In addition, other data will be extracted such as any shapefiles, 1D netCDF tables, etc. The input to the tool should be a .zip file that was created using the WRF Hydro pre-processing tools. The tool will create the output folder if it does not already exist, and write all results to that location.

#### Request help message from the script
This tool has a single input and single output parameter. Using the command line, we can take a look at the arguments.

In [11]:
! python Examine_Outputs_of_GIS_Preprocessor.py -h

Script initiated at Tue Nov  3 00:15:13 2020
usage: Examine_Outputs_of_GIS_Preprocessor.py [-h] -i IN_ZIP -o OUT_FOLDER

This tool takes the output zip file from the ProcessGeogrid script and creates
a raster from each output NetCDF file. The Input should be a .zip file that
was created using theWRF Hydro pre-processing tools. The tool will create the
folder which will contain theresults (out_folder), if that folder does not
already exist.

optional arguments:
  -h, --help     show this help message and exit
  -i IN_ZIP      Path to WRF Hydro routing grids zip file.
  -o OUT_FOLDER  Path to output folder.


----
#### Execute the script using command-line syntax
We will define the output directory for the tool to create, then use defined variables to execute the command line tool.

In [12]:
# Define output directory to store GeoTiff output of all routing stack grids
raster_outputs = os.path.join(output_folder, "Raster_Outputs")

# Print information to screen for reference
print('Command to run:\n')
print('python Examine_Outputs_of_GIS_Preprocessor.py \\\n\t -i {0} \\\n\t -o {1}\n'.format(out_zip, raster_outputs))

# Run the script with required parameters
! python Examine_Outputs_of_GIS_Preprocessor.py -i {out_zip} -o {raster_outputs}

Command to run:

python Examine_Outputs_of_GIS_Preprocessor.py \
	 -i /home/docker/GIS_Training/Outputs/croton_test.zip \
	 -o /home/docker/GIS_Training/Outputs/Raster_Outputs

Script initiated at Tue Nov  3 00:15:14 2020
    File Copied: lakes.prj
    File Copied: lakes.shx
  GeoTransform: (1841999.0194101864, 1000.0, 0.0, 278495.814356732, 0.0, -1000.0)
  DX: 1000.0
  DY: 1000.0
  PROJ.4 string: +proj=lcc +lat_0=41.4710083007812 +lon_0=-97 +lat_1=30 +lat_2=60 +x_0=0 +y_0=0 +R=6370000 +units=m +no_defs
    File Created: /home/docker/GIS_Training/Outputs/Raster_Outputs/BASIN.tif
    File Copied: LAKEPARM.nc
    File Copied: lakes.dbf
    File Copied: GEOGRID_LDASOUT_Spatial_Metadata.nc
    File Copied: lakes.shp
  GeoTransform: (1841999.0194101864, 250.0, 0.0, 278495.814356732, 0.0, -250.0)
  DX: 250.0
  DY: 250.0
  PROJ.4 string: +proj=lcc +lat_0=41.4710083007812 +lon_0=-97 +lat_1=30 +lat_2=60 +x_0=0 +y_0=0 +R=6370000 +units=m +no_defs
    File Created: /home/docker/GIS_Training/Outpu

### 6. [Optional] Build Non-NWM WRF-Hydro Configurations of the Croton, NY Test Case

We will run command-line arguments to create output directories for each of the test cases. Then, each cell will run the WRF-Hydro GIS Pre-processor to build the 'routing stack' files. The final command is used to unzip the outputs so they can be more readily used by WRF-Hydro. There are 4 Non-NWM test-case configurations which can all be created using different combinations of arguments to the GIS pre-processor. Currently missing at this time is the Reach-with-Lakes capability as well as the ability to build wrfinput, hydro2dtbl, and soil_properties files.

#### A. Gridded channel routing configuration with reservoirs and forecast points

This domain is a gridded domain with lakes, gages and a regridding factor of 4 and threshold of 20

In [16]:
# Create the output directory and ensure it is empty
! mkdir -p /home/docker/GIS_Training/Outputs/Gridded
! rm -rf /home/docker/GIS_Training/Outputs/Gridded/*

# Run the GIS Pre-processing script (with line-breaks)
! python Build_Routing_Stack.py \
    -i /home/docker/wrf-hydro-training/example_case/Gridded/DOMAIN/geo_em.d01.nc \
    -l /home/docker/GIS_Training/Croton_Lambert/lake_shapes/lakes.shp \
    --CSV /home/docker/GIS_Training/Croton_Lambert/croton_frxst_pts_FOSS.csv \
    -d /home/docker/GIS_Training/Croton_Lambert/NED_30m_Croton.tif \
    -R 4 \
    -t 20 \
    -o /home/docker/GIS_Training/Outputs/Gridded/Gridded_r4_t20_lakes_frxst_mask.zip

# Unzip the directory in-place
! unzip /home/docker/GIS_Training/Outputs/Gridded/Gridded_r4_t20_lakes_frxst_mask.zip -d /home/docker/GIS_Training/Outputs/Gridded

Script initiated at Tue Nov  3 00:15:17 2020
  Parameter values that have not been altered from script default values:
    Using default basin mask setting: False
    Using default reach-based routing setting: False
    Using default OVROUGHRTFAC parameter value: 1.0
    Using default RETDEPRTFAC parameter value: 1.0
  Values that will be used in building this routing stack:
    Input WPS Geogrid file: /home/docker/wrf-hydro-training/example_case/Gridded/DOMAIN/geo_em.d01.nc
    Forecast Point CSV file: /home/docker/GIS_Training/Croton_Lambert/croton_frxst_pts_FOSS.csv
    Mask CHANNELGRID variable to forecast basins?: False
    Create reach-based routing (RouteLink) files?: False
    Lake polygon feature class: /home/docker/GIS_Training/Croton_Lambert/lake_shapes/lakes.shp
    Input high-resolution DEM: /home/docker/GIS_Training/Croton_Lambert/NED_30m_Croton.tif
    Regridding factor: 4
    Stream initiation threshold: 20
    OVROUGHRTFAC parameter value: 1.0
    RETDEPRTFAC parameter

#### B. Gridded channel routing configuration with forecast points, and channel masking

This domain is a gridded domain without lakes. It contains gages, masked basins and a regridding factor of 4 and thrshold of 20

In [19]:
# Create the output directory and ensure it is empty
! mkdir -p /home/docker/GIS_Training/Outputs/Gridded_no_lakes
! rm -rf /home/docker/GIS_Training/Outputs/Gridded_no_lakes/*

# Run the GIS Pre-processing script (with line-breaks)
! python Build_Routing_Stack.py \
    -i /home/docker/wrf-hydro-training/example_case/Gridded_no_lakes/DOMAIN/geo_em.d01.nc \
    --CSV /home/docker/GIS_Training/Croton_Lambert/croton_frxst_pts_FOSS.csv \
    -d /home/docker/GIS_Training/Croton_Lambert/NED_30m_Croton.tif \
    -R 4 \
    -t 20 \
    -o /home/docker/GIS_Training/Outputs/Gridded_no_lakes/Gridded_r4_t20_frxst_mask.zip

# Unzip the directory in-place
! unzip /home/docker/GIS_Training/Outputs/Gridded_no_lakes/Gridded_r4_t20_frxst_mask.zip -d /home/docker/GIS_Training/Outputs/Gridded_no_lakes/

Script initiated at Tue Nov  3 00:15:41 2020
  Parameter values that have not been altered from script default values:
    Using default basin mask setting: False
    Using default reach-based routing setting: False
    Using default OVROUGHRTFAC parameter value: 1.0
    Using default RETDEPRTFAC parameter value: 1.0
  Values that will be used in building this routing stack:
    Input WPS Geogrid file: /home/docker/wrf-hydro-training/example_case/Gridded_no_lakes/DOMAIN/geo_em.d01.nc
    Forecast Point CSV file: /home/docker/GIS_Training/Croton_Lambert/croton_frxst_pts_FOSS.csv
    Mask CHANNELGRID variable to forecast basins?: False
    Create reach-based routing (RouteLink) files?: False
    Lake polygon feature class: 
    Input high-resolution DEM: /home/docker/GIS_Training/Croton_Lambert/NED_30m_Croton.tif
    Regridding factor: 4
    Stream initiation threshold: 20
    OVROUGHRTFAC parameter value: 1.0
    RETDEPRTFAC parameter value: 1.0
    Input channel initiation start point 

#### C. Reach-based channel routing configuration with forecast points

This domain is a reach-based routing configuration with gages, no masking, no lakes. Regridding factor of 4 and threshold of 20

In [18]:
# Create the output directory and ensure it is empty
! mkdir -p /home/docker/GIS_Training/Outputs/Reach
! rm -rf /home/docker/GIS_Training/Outputs/Reach/*

# Run the GIS Pre-processing script (with line-breaks)
! python Build_Routing_Stack.py \
    -i /home/docker/wrf-hydro-training/example_case/Reach/DOMAIN/geo_em.d01.nc \
    -r True \
    --CSV /home/docker/GIS_Training/Croton_Lambert/croton_frxst_pts_FOSS.csv \
    -d /home/docker/GIS_Training/Croton_Lambert/NED_30m_Croton.tif \
    -R 4 \
    -t 20 \
    -o /home/docker/GIS_Training/Outputs/Reach/Reach_r4_t20_frxst.zip

# Unzip the directory in-place
! unzip /home/docker/GIS_Training/Outputs/Reach/Reach_r4_t20_frxst.zip -d /home/docker/GIS_Training/Outputs/Reach/

Script initiated at Tue Nov  3 00:15:25 2020
  Parameter values that have not been altered from script default values:
    Using default basin mask setting: False
    Using default OVROUGHRTFAC parameter value: 1.0
    Using default RETDEPRTFAC parameter value: 1.0
  Values that will be used in building this routing stack:
    Input WPS Geogrid file: /home/docker/wrf-hydro-training/example_case/Reach/DOMAIN/geo_em.d01.nc
    Forecast Point CSV file: /home/docker/GIS_Training/Croton_Lambert/croton_frxst_pts_FOSS.csv
    Mask CHANNELGRID variable to forecast basins?: False
    Create reach-based routing (RouteLink) files?: True
    Lake polygon feature class: 
    Input high-resolution DEM: /home/docker/GIS_Training/Croton_Lambert/NED_30m_Croton.tif
    Regridding factor: 4
    Stream initiation threshold: 20
    OVROUGHRTFAC parameter value: 1.0
    RETDEPRTFAC parameter value: 1.0
    Input channel initiation start point feature class: None
    Input groundwater basin polygons: None
  

### You have reached the end of Lesson S2